In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from ipywidgets import interactive, VBox, HBox, Output
from IPython.display import display


def tao_dom_gaussian(ma_tran, vi_tri, cuong_do, do_rong):
    """
    Vẽ một "đốm sáng" Gaussian 2D lên ma trận tại một vị trí.
    """
    k_thuoc = ma_tran.shape[0]
    y, x = vi_tri
    ys, xs = np.ogrid[:k_thuoc, :k_thuoc]
    
    khoang_cach_binh_phuong = (xs - x)**2 + (ys - y)**2
    dom_sang = cuong_do * np.exp(-khoang_cach_binh_phuong / (2 * do_rong**2))
    
    ma_tran += dom_sang
    return ma_tran

def gia_lap_ban_do_tinh(goc_MCP, goc_PIP, goc_DIP, k_thuoc=8):
    """
    Giả lập bản đồ áp suất 8x8 TĨNH (ĐÃ SỬA LỖI ĐỘNG HỌC)
    Bao gồm 4 điểm áp lực: 3 khớp và 1 đầu ngón tay.
    """
    ban_do = np.zeros((k_thuoc, k_thuoc))
    
    # Chiều dài (tỷ lệ) của các đốt
    L1_scaled = 2.0  # Khoảng cách từ MCP đến PIP khi duỗi
    L2_scaled = 2.0  # Khoảng cách từ PIP đến DIP khi duỗi
    L3_scaled = 1.0  # Khoảng cách từ DIP đến Đầu ngón khi duỗi
    
    # Tọa độ cơ sở
    y_MCP = 7.0 # Bắt đầu từ hàng gần cuối
    x_MCP = 4.0 # Giữ ở cột giữa
    
    rad_MCP = np.deg2rad(goc_MCP)
    rad_PIP = np.deg2rad(goc_PIP)
    rad_DIP = np.deg2rad(goc_DIP)

    # 1. Khớp MCP (nối lòng bàn tay)
    # Áp suất giảm đi khi gập (vì nó nhấc lên khỏi bề mặt)
    cuong_do_MCP = 0.8 * np.cos(rad_MCP) 
    tao_dom_gaussian(ban_do, (y_MCP, x_MCP), cuong_do=cuong_do_MCP, do_rong=1.5)
    
    # 2. Khớp PIP (khớp giữa)
    # y_PIP di chuyển từ (7 - L1) đến 7
    y_PIP = y_MCP - L1_scaled * np.cos(rad_MCP)
    cuong_do_PIP = 1.0 # Khớp này luôn tạo áp suất
    tao_dom_gaussian(ban_do, (y_PIP, x_MCP), cuong_do=cuong_do_PIP, do_rong=1.2)
    
    # 3. Khớp DIP (khớp xa)
    goc_tich_luy_PIP = rad_MCP + rad_PIP
    y_DIP = y_PIP - L2_scaled * np.cos(goc_tich_luy_PIP)
    cuong_do_DIP = 0.8
    tao_dom_gaussian(ban_do, (y_DIP, x_MCP), cuong_do=cuong_do_DIP, do_rong=1.0)
    
    # 4. ĐẦU NGÓN TAY (Fingertip) - (Như bạn đề xuất)
    goc_tich_luy_DIP = goc_tich_luy_PIP + rad_DIP
    y_Tip = y_DIP - L3_scaled * np.cos(goc_tich_luy_DIP)
    # Áp suất ở đầu ngón tay mạnh nhất khi duỗi
    cuong_do_Tip = 0.5 + 0.5 * (1 - goc_DIP/90.0)
    tao_dom_gaussian(ban_do, (y_Tip, x_MCP), cuong_do=cuong_do_Tip, do_rong=1.0)
    
    # Thêm nhiễu ngẫu nhiên
    nhieu = np.random.rand(k_thuoc, k_thuoc) * 0.1
    ban_do += nhieu
    
    # Chuẩn hóa
    if ban_do.max() > 0:
        ban_do = ban_do / ban_do.max()
        
    return ban_do

def gia_lap_ban_do_dong(vi_tri_tham_do, do_cung, k_thuoc=8):
    """
    Giả lập bản đồ áp suất ĐỘNG (phản lực) khi thăm dò.
    Độ cứng (0-10) càng cao, đỉnh áp suất càng cao và nhọn.
    """
    ban_do = np.zeros((k_thuoc, k_thuoc))
    
    cuong_do = do_cung / 10.0 
    do_rong = 2.0 - (do_cung / 10.0) * 1.5
    
    tao_dom_gaussian(ban_do, vi_tri_tham_do, cuong_do, do_rong)
    
    nhieu = np.random.rand(k_thuoc, k_thuoc) * 0.05
    ban_do += nhieu
    
    if ban_do.max() > 0:
        ban_do = ban_do / ban_do.max()
        
    return ban_do

def ve_khung_xuong_3d(ax, goc_MCP, goc_PIP, goc_DIP, mau='b', label=''):
    """
    Vẽ một mô hình khung xương 3D đơn giản của ngón tay.
    """
    L1 = 3 # Đốt gần (MCP -> PIP)
    L2 = 2 # Đốt giữa (PIP -> DIP)
    L3 = 1 # Đốt xa (DIP -> Đầu ngón)
    
    rad_MCP = np.deg2rad(goc_MCP)
    rad_PIP = np.deg2rad(goc_PIP)
    rad_DIP = np.deg2rad(goc_DIP)

    J0 = [0, 0, 0]
    
    J1_y = L1 * np.cos(rad_MCP)
    J1_z = L1 * np.sin(rad_MCP)
    J1 = [0, J1_y, J1_z]
    
    goc_tich_luy_PIP = rad_MCP + rad_PIP
    J2_y = J1[1] + L2 * np.cos(goc_tich_luy_PIP)
    J2_z = J1[2] + L2 * np.sin(goc_tich_luy_PIP)
    J2 = [0, J2_y, J2_z]
    
    goc_tich_luy_DIP = goc_tich_luy_PIP + rad_DIP
    Tip_y = J2[1] + L3 * np.cos(goc_tich_luy_DIP)
    Tip_z = J2[2] + L3 * np.sin(goc_tich_luy_DIP)
    Tip = [0, Tip_y, Tip_z]
    
    points = np.array([J0, J1, J2, Tip])
    
    ax.plot(points[:, 0], points[:, 1], points[:, 2], marker='o', color=mau, label=label)
    ax.set_xlabel('X')
    ax.set_ylabel('Y (Dọc ngón tay)')
    ax.set_zlabel('Z (Gập)')
    ax.set_title("Mô phỏng Mô hình Hóa Ngón tay 3D")
    ax.set_xlim([-1, 1])
    ax.set_ylim([0, 6])
    ax.set_zlim([0, 6])

# --- HÀM CẬP NHẬT GIAO DIỆN CHÍNH (ĐÃ NÂNG CẤP) ---

def cap_nhat_giao_dien(goc_MCP, goc_PIP, goc_DIP, do_cung):
    """
    Hàm này được gọi mỗi khi bạn kéo BẤT KỲ thanh trượt nào.
    Nó sẽ vẽ lại cả BA biểu đồ.
    """
    # 1. Tạo một khung hình mới với 3 ô (1 hàng, 3 cột)
    fig, (ax1, ax2, ax3_placeholder) = plt.subplots(1, 3, figsize=(18, 6))
    
    # --- Vẽ Ô 1: Bản đồ Áp suất Tĩnh (Chỉ phụ thuộc 3 góc) ---
    ban_do_tinh = gia_lap_ban_do_tinh(goc_MCP, goc_PIP, goc_DIP)
    ax1.imshow(ban_do_tinh, cmap='viridis', vmin=0, vmax=1)
    ax1.set_title(f"Bản đồ Tĩnh (Hình dạng)")
    ax1.set_xticks(np.arange(8))
    ax1.set_yticks(np.arange(8))

    # --- Vẽ Ô 2: Bản đồ Áp suất Động (Chỉ phụ thuộc Độ cứng) ---
    # Giả sử chúng ta luôn "thăm dò" tại trung tâm (4, 4)
    vi_tri_tham_do = (4, 4)
    ban_do_dong = gia_lap_ban_do_dong(vi_tri_tham_do, do_cung)
    ax2.imshow(ban_do_dong, cmap='inferno', vmin=0, vmax=1)
    ax2.set_title(f"Bản đồ Động (Phản lực {do_cung:.1f}/10)")
    ax2.set_xticks(np.arange(8))
    ax2.set_yticks(np.arange(8))

    # --- Vẽ Ô 3: Mô hình 3D (Kết hợp) ---
    fig.delaxes(ax3_placeholder)
    ax3 = fig.add_subplot(1, 3, 3, projection='3d')
    
    # Vẽ ngón tay duỗi thẳng (để so sánh)
    ve_khung_xuong_3d(ax3, 0, 0, 0, mau='g', label='Duỗi thẳng (0°)')
    
    # Vẽ ngón tay co (theo 3 góc)
    # Gán nhãn (theo độ cứng)
    label_do_cung = f'Co cứng (Độ cứng {do_cung:.1f})'
    ve_khung_xuong_3d(ax3, goc_MCP, goc_PIP, goc_DIP, mau='r', label=label_do_cung)
    
    ax3.legend()
    
    plt.tight_layout()
    plt.show()

# --- TẠO CÁC THANH TRƯỢT (4 CÁI) ---

slider_MCP = widgets.IntSlider(value=30, min=0, max=90, step=1, description='Góc MCP:')
slider_PIP = widgets.IntSlider(value=50, min=0, max=90, step=1, description='Góc PIP:')
slider_DIP = widgets.IntSlider(value=10, min=0, max=90, step=1, description='Góc DIP:')

# Thêm thanh trượt mới cho Độ Cứng
slider_do_cung = widgets.FloatSlider(value=8.5, min=0.0, max=10.0, step=0.1, 
                                     description='Độ cứng:',
                                     readout_format='.1f')

# --- SẮP XẾP GIAO DIỆN VÀ HIỂN THỊ ---

# 1. Liên kết 4 thanh trượt với hàm cap_nhat_giao_dien
interactive_output = interactive(cap_nhat_giao_dien, 
                                 goc_MCP=slider_MCP, 
                                 goc_PIP=slider_PIP, 
                                 goc_DIP=slider_DIP,
                                 do_cung=slider_do_cung)

# 2. Tách riêng phần điều khiển (sliders) mà BẠN muốn
ui_controls = VBox([
    HBox([slider_MCP, slider_PIP]), 
    HBox([slider_DIP, slider_do_cung])
])

# 3. Tách riêng phần KẾT QUẢ (biểu đồ) từ hàm interactive
# (interactive_output.children[-1] chính là ô output)
output_charts = interactive_output.children[-1]

# 4. Hiển thị theo đúng thứ tự bạn muốn: Điều khiển ở trên, Kết quả ở dưới
display(VBox([ui_controls, output_charts]))